In [3]:
%matplotlib widget

In [4]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

import random

import csv
import json
from PIL import Image

from scipy import signal
from scipy import misc

import math

import image_utils

import time

Image.MAX_IMAGE_PIXELS = None

In [5]:
strings = ["0486052bb",
    "095bf7a1f",
    "1e2425f28",
    "2f6ecfcdf",
    "54f2eec69",
    "aaa6a05cc",
    "cb2d976f4",
    "e79de561c",
    ]
db = "./DB"
sample_id = "e79de561c"
sample = db + "/train/" + sample_id
sample_image = Image.open(sample+".tiff")
with open(sample+".json") as sample_json_file:
    sample_json = json.load(sample_json_file)

In [6]:
fig, ax = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def evaluate_single_FT_Kernel(span, FT_amp):
    """
    Center: The [x, y] coordinates where the center of the FT-Kernel will be
    Span:   The 'radius' of the bouding region where the FT-Kernel will be evaluated
    FT_Amp: A 2xN array of amplitudes representing sine waves that oscillate n times within the span
    
    TODO: Rebuild so each 'pixel' is the integral of the sine wave it overlaps
    """
    # Building the Kernel
    h_linspace = np.linspace(0, 2*np.pi, num=span+1)
    v_linspace = h_linspace

    
    def subcell_evaluation(freq, amp, samples):
        # Evaluate the integral of the sine wave over the given linear space
        # Integral of sine is negative cosine
        # Integrate from halfpitch below to halfpitch above
        
        lower_sum = -np.cos(samples[:-1]*(freq+1)*0.5)
        upper_sum = -np.cos(samples[1:]*(freq+1)*0.5)
        
        result = (upper_sum - lower_sum)*amp
        return result
        

        
    
    # lf_lambda = lambda freq, amp: np.sin(h_linspace*(freq+1)*0.5) * amp
    
    h_contrib = np.sum(np.stack([subcell_evaluation(freq, amp, h_linspace) for freq, amp in enumerate(FT_amp[0, :span//2])]), axis=0)
    h_region = np.stack([h_contrib]*span)
    print(h_region.shape)
    

    v_contrib = np.sum(np.stack([subcell_evaluation(freq, amp, v_linspace) for freq, amp in enumerate(FT_amp[1, :span//2])]), axis=0)
    v_region = np.stack([v_contrib]*span).T
    print(v_region.shape)
    
    
    ft_kernel = h_region + v_region

    
    
    return ft_kernel

In [8]:
span = 3
FT_amp = np.array([[0, 1],
                   [0, 1]])
ft_kernel = evaluate_single_FT_Kernel(span, FT_amp)
print(ft_kernel.shape)

(3, 3)
(3, 3)
(3, 3)


In [9]:
ax.matshow(ft_kernel)

In [10]:
image_array = np.asarray(sample_image)

In [11]:
print(image_array.shape)

(16180, 27020, 3)


In [10]:
start_time = time.time()
kif = signal.convolve2d(image_array[:, :, 0], ft_kernel)
duration = time.time() - start_time
print(duration)

24.16814923286438


In [12]:
print(kif.shape)

NameError: name 'kif' is not defined

In [13]:
fig2, ax2 = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# ax2.imshow(kif)

NameError: name 'kif' is not defined